In [3]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D,Lambda
from keras.layers import MaxPooling2D,UpSampling2D,Concatenate,Conv2DTranspose
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.models import load_model
#from keras_retinanet.models import load_model
#import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.callbacks import LearningRateScheduler,Callback
from keras.optimizers import RMSprop
import cv2
import csv
from math import *
from keras.applications.vgg16 import VGG16 
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
import h5py
from keras.utils import to_categorical
from IPython.display import clear_output
from sklearn.model_selection import train_test_split
from sklearn import datasets,svm
from keras.preprocessing.image import ImageDataGenerator
from skimage import measure,img_as_float32
from imutils import contours
import imutils
import scipy.misc as sm
from PIL import Image
from timeit import default_timer as timer
from numba import *
import os
#from keras_retinanet.models.resnet import custom_objects
%matplotlib inline

#########################################################################################################################


In [11]:
lst_tp=[]
with open('Centers/2 (copy).txt','r') as f:
    reader = csv.reader(f,dialect = 'excel',delimiter=' ')
    for row in reader:
        x = row[0]
        y = row[1]
        lst_tp.append([x,y])

In [12]:
lst=[]
with open('Centers/2_FPN_res_x3.txt','r') as f:
    reader = csv.reader(f,dialect = 'excel',delimiter=' ')
    for row in reader:
        x = row[0]
        y = row[1]
        lst.append([x,y])

In [13]:
print(len(lst_tp))
print(len(lst))

971
954


In [14]:
cnt = 0
for i in range(len(lst)):
    x = int(lst[i][0])
    y = int(lst[i][1])
    
    for j in range(len(lst_tp)):
        x_tp = int(lst_tp[j][0])
        y_tp = int(lst_tp[j][1])
        
        if x_tp<=(x+5) and x_tp>=(x-5) and y_tp<=(y+5) and y_tp>=(y-5):
            cnt = cnt + 1
            #del lst[i]
            lst[i] = [-1,-1]
            del lst_tp[j]
            break

print(cnt)
print(len(lst)-cnt)
print(len(lst_tp))
print(lst)

831
123
140
[[-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], ['80', '873'], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], ['173', '3332'], ['175', '3261'], [-1, -1], ['180', '3524'], [-1, -1], [-1, -1], [-1, -1], [-1, -1], ['210', '1476'], [-1, -1], ['218', '774'], [-1, -1], [-1, -1], ['233', '722'], [-1, -1], [-1, -1], [-1, -1], [-1, -1], ['274', '2986'], [-1, -1], [-1, -1], [-1, -1], [-1, -1], ['303

In [15]:
im_p = "Images_for_FCRN/Test/Test_Results/2p.JPG"
img = cv2.imread(im_p)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = np.asarray(img)
print(img[0][100][1])

1


In [16]:
c = [255,0,0]
for i in range(len(lst_tp)):
    y = int(lst_tp[i][0])
    x = int(lst_tp[i][1])
    
    for i in range(10):
        img[x+i][y]=c
        img[x][y+i]=c
        img[x+i][y+i]=c
        img[x-i][y]=c
        img[x][y-i]=c
        img[x-i][y-i]=c

d = [0,255,0]
for i in range(len(lst)):
    print(i)
    y = int(lst[i][0])
    x = int(lst[i][1])
    
    if x==-1 and y==-1:
        continue
    for i in range(10):
        img[x+i][y]=d
        img[x][y+i]=d
        img[x+i][y+i]=d
        img[x-i][y]=d
        img[x][y-i]=d
        img[x-i][y-i]=d

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
sm.imsave('evaluation/eval_2_fpnx3.jpg',img)

/home/sumeet/.virtualenvs/dl4cv/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
  """Entry point for launching an IPython kernel.
